In [1]:
import pandas as pd;
import numpy as np;
import cv2;
import matplotlib.pyplot as plt;
import matplotlib;
import os;
from collections import Counter;
import random; 
import time;

%matplotlib inline

In [2]:

os.listdir('C:/Users/adity/Documents/Projects/')
#Reading the data 
# os.listdir("C:\\Users\\adity\\Documents\\Projects")

maindir="C:/Users/adity/Documents/Projects/"
trainDir="train"
path=os.path.join(maindir,trainDir);
X=[];
y=[];
for p in os.listdir(path):
    class_val=p.split(".")[0];
    img_array=cv2.imread(os.path.join(path,p),cv2.IMREAD_GRAYSCALE)
    new_img_array=cv2.resize(img_array,dsize=(80,240));
#     plt.imshow(new_img_array)
    X.append(new_img_array);
    y.append(class_val);
X=np.array(X).reshape(-1,80,80,3);
X_flatten=X.reshape(X.shape[0],-1).T
y_flatten=np.array(y)

# testDir="test1"
# path=os.path.join(maindir,testDir);
# X_test=[];
# y_test=[];
# #Reading the Test Data.
# for p in os.listdir(path):
#     class_val=p.split(".");
#     img_array=cv2.imread(os.path.join(path,p),cv.IMREAD_GRAYSCALE)
#     new_img_array=cv.resize(img_array,dsize=(80,80))
#     X_test.append(new_img_array);
#     y_test.append(class_val);
# X_test=np.array(X_test).reshape(-1,80,80,1)
# X_test_flatten=X_test.reshape(X_test.shape[0],-1).T
# y_test=np.array(y_test)
    

In [3]:
y_train=y_flatten.reshape(y_flatten.shape[0],1)
X_train=X_flatten.copy()

print('Shape of X in Training DataSet ',X_train.shape)
print('Shape of y in Training Dataset',y_train.shape)
del X

Shape of X in Training DataSet  (19200, 25000)
Shape of y in Training Dataset (25000, 1)


###  Image Data Preprocessing.

* Also, class cat will be assigned 1 and dog 0. ( Although I love dogs over cats ). 

In [4]:
#Splitting the data into training and validation portions 
X_train=X_train/255.
data=pd.DataFrame(X_train).T
data.shape
data['Class']=y_train
data_cat=data[data['Class']=='cat'].reset_index()
data_dog=data[data['Class']=='dog'].reset_index()
indices=random.sample(range(12500),12500)
data_cat_train=data_cat.iloc[indices[:11250],:]
data_cat_validation=data_cat.iloc[indices[11250:],:]
data_dog_train=data_dog.iloc[indices[:11250],:]
data_dog_validation=data_dog.iloc[indices[11250:],:]

# data_cat_train.shape
data_train=pd.concat([data_cat_train,data_dog_train])
data_validation=pd.concat([data_cat_validation,data_dog_validation])

data_train=data_train.reset_index().iloc[:,1:]
data_train['NUM_CLASS']=0
data_train.loc[data_train['Class']=='cat','NUM_CLASS']=1
data_train=data_train.drop(['Class','index'],axis=1)
data_train=data_train.T
data_validation=data_validation.reset_index().iloc[:,1:]
data_validation['NUM_CLASS']=0
data_validation.loc[data_validation['Class']=='cat','NUM_CLASS']=1
data_validation=data_validation.drop(['Class','index'],axis=1)
data_validation=data_validation.T

In [5]:
print('Data Train Shape',data_train.shape)
print('Data Validation Shape',data_validation.shape)



Data Train Shape (19201, 22500)
Data Validation Shape (19201, 2500)


In [6]:
def layer_sizes(X,y):
    ''' Function to set the layer sizes '''
    n_x=X.shape[0];
    n_h=5;
    n_y=y.shape[0]
    
    return n_x,n_h,n_y;

    

In [7]:
def initialize_parameters (n_x,n_h,n_y):
    ''' Initializing the parameters to random values.'''
    W1=np.random.randn(n_h,n_x)*0.001;
    b1=np.zeros((n_h,1));
    W2=np.random.randn(n_y,n_h)*0.001;
    b2=np.zeros((n_y,1));
    
    assert (W1.shape==(n_h,n_x))
    assert (b1.shape==(n_h,1))
    assert (W2.shape==(n_y,n_h))
    assert (b2.shape==(n_y,1))
    
    
    return {'W1':W1,'b1':b1,'W2':W2,'b2':b2};


In [8]:

def sigmoid (z):
    ''' Implementation of sigmoid '''
    a=1/(1+np.exp(-z));
    return a; 


def tangent_h(z):
    '''Implementation tanh function''' 
    return np.tanh(z);

In [9]:
parameters = initialize_parameters(5,5, 1)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 5.88794441e-04  1.62250584e-05  6.41015163e-04  7.95197898e-04
   1.01986584e-03]
 [-4.08012014e-04  5.00426509e-04 -1.20988682e-03  1.67710125e-04
   6.63342391e-04]
 [ 7.95612628e-04  1.93913624e-04 -1.77102281e-04  1.20272645e-03
   2.51268387e-05]
 [ 6.36772847e-04 -3.91593012e-04  7.29299324e-04  9.40488511e-04
  -4.16203559e-04]
 [-1.25389272e-03  1.57215672e-03  1.96739520e-03  3.15506627e-04
  -2.03418096e-04]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
W2 = [[ 1.71553304e-03  1.98934321e-03  2.43623027e-03 -1.28925681e-03
   6.55012464e-05]]
b2 = [[0.]]


In [10]:
def forward_propagation(X,parameters):
    start = time.time()
    ''' Implementing forward propagation'''
    W1=parameters['W1'];
    b1=parameters['b1'];
    W2=parameters['W2'];
    b2=parameters['b2'];
#     print('Shape W1',W1.shape, 'W2 shape',W2.shape);
    #Forward prop 
    Z1=np.dot(W1,X)+b1;
#     A1=sigmoid(Z1);
    A1=tangent_h(Z1);
    Z2=np.dot(W2,A1)+b2;
    A2=sigmoid(Z2);
#     A2=tangent_h(Z2);
#     print('A2 shape',A2.shape,' X.shape ',X.shape)
    assert(A2.shape[1]== X.shape[1]);
    end = time.time()
#     print('Execution time for Forward Propagation',end - start)
    return {'Z1':Z1,'A1':A1,'Z2':Z2,'A2':A2};
    
    

In [11]:
def compute_cost(A2,Y):
    '''Computing cost values for an iteration'''
    start = time.time()
    m=Y.shape[1];
#     cost=(-1/m)* np.sum( np.multiply(np.log(A2),Y),np.multiply(np.log(1-A2),(1-Y)));
#     cost=float(np.squeeze(cost));
    
#     m=Y_train.shape[1];
    pt1=np.multiply(np.log(A2),Y)
    pt2=np.multiply(np.log(1-A2),(1-Y))
    cost=np.sum(pt1+pt2)*(-1/m)
    cost=(np.squeeze(cost))
    assert(isinstance(cost,float));
    end = time.time()
#     print('Execution time for Compute Cost',end - start)
    return cost;
    

In [12]:
def backward_propagation(cache,parameters,X,Y):
    ''' Backward Propagation Implementation for the ANN'''
    start = time.time()
    m=X.shape[1];
    W1=parameters['W1'];
    b1=parameters['b1'];
    W2=parameters['W2'];
    b2=parameters['b2'];
    
    A1=cache['A1'];
    A2=cache['A2'];
    Z1=cache['Z1'];
    
    dz2=A2-Y;
    dw2=(1/m)*np.dot(dz2,A1.T)
    db2=(1/m)*(np.sum(dz2,axis=1,keepdims=True)) 
#     dz1=np.multiply (np.dot(W2.T,dz2),(sigmoid(Z1)*(1-sigmoid(Z1))));
    dz1=np.multiply (np.dot(W2.T,dz2),(1-(np.power(np.tanh(Z1),2))));
    dw1=(1/m)*np.dot(dz1,X.T);
    db1=(1/m)* np.sum(dz1,axis=1,keepdims=True)
    grads={'dw1':dw1,'db1':db1,'dw2':dw2,'db2':db2}    
    end = time.time()
#     print('Execution time for Backward Propagation',end - start)

    return grads;



In [13]:
def update_parameters(grads,parameters,learning_rate):
    '''Update the parameters '''
    #parameters
    start = time.time()
    W1=parameters['W1'];
    b1=parameters['b1'];
    W2=parameters['W2'];
    b2=parameters['b2'];
    #grads
    dw1=grads['dw1'];
    dw2=grads['dw2'];
    db1=grads['db1'];
    db2=grads['db2'];
    
    #update 
    W1=W1 - (learning_rate*dw1);
    b1=b1-(learning_rate*db1);
    W2=W2-(learning_rate*dw2);
    b2=b2-(learning_rate*db2);
    
    parameters={'W1':W1,'W2':W2,'b1':b1,'b2':b2};
    end = time.time()
#     print('Execution time for Update Parameters',end - start)

    return parameters;
    
    
              


In [ ]:
def nn_model ( X,Y,n_h,num_iterations,learning_rate=0.001,print_cost=False):
    '''Combining all the functions to generate a neural network.'''
    n_x=X.shape[0]; # Since the matrix shape is nX,M where nX is the number of features and M is the number of examples.
    n_y=Y.shape[0]; 
#     print(n_x,' ',n_y)
    
    #Initialize Parameters
    parameters= initialize_parameters(n_x,n_h,n_y);
    
    
    #Iteration Start (Gradient Descent)
    for i in range(0,num_iterations):
        start = time.time()
        #Forward Propagation
        cache = forward_propagation(X,parameters);
#         print('A2', cache['A2'])
#         print('Y',Y)
        cost= compute_cost(cache['A2'],Y); # Generates the cost 
        #Do Backward Propagation
        grads= backward_propagation(cache,parameters,X,Y);
        #Update Weights.
        parameters=update_parameters(grads,parameters,learning_rate);
        end = time.time()
#     print('Execution time for One Iteration',end - start)
#         if print_cost  and  i%1000==0:
        print('Cost',cost)
        print("Cost after iteration %i : %f"%(i,cost));
        
    return parameters;
    

In [ ]:
def predict(parameters,X):
    cache=forward_propagation(X,parameters);
    A2=cache['A2'];
    predictions=np.where(A2>=0.5,1,0);
    return predictions;
    

In [14]:
data_train.shape

X_train=data_train.T.iloc[:,:-1].T.to_numpy()
Y_train=data_train.T.iloc[:,-1].to_numpy()
Y_train=Y_train.reshape((1,Y_train.shape[0]))
print('X train shape ',X_train.shape)
print('X train shape ',Y_train.shape)

X train shape  (19200, 22500)
X train shape  (1, 22500)


In [15]:
data_validation.shape

X_validation=data_validation.T.iloc[:,:-1].T.to_numpy()
Y_validation=data_validation.T.iloc[:,-1].to_numpy()
Y_validation=Y_validation.reshape((1,Y_validation.shape[0]))
print('X validation shape ',X_validation.shape)
print('X validation shape ',Y_validation.shape)

X validation shape  (19200, 2500)
X validation shape  (1, 2500)


In [ ]:
# parameters=nn_model(X_train,Y_train.T,5,10000,0.3,True);

In [16]:
n_x=X_train.shape[0]; # Since the matrix shape is nX,M where nX is the number of features and M is the number of examples.
n_y=Y_train.shape[0]; 
n_h=10;
parameters= initialize_parameters(n_x,n_h,n_y);
start = time.time()
for i in range(0,10000):
    cache = forward_propagation(X_train,parameters);
    # cache['A1'].shape
    cost= compute_cost(cache['A2'],Y_train); # Generates the cost 
#     print('Cost',cost)
    grads= backward_propagation(cache,parameters,X_train,Y_train);
    parameters=update_parameters(grads,parameters,0.3);
    if i%1000==0:
        end = time.  time()
        print("Cost after iteration %i : %f"%(i,cost));
        print('Execution time for 1000 Iteration',end - start)
        start=time.time()



Cost after iteration 0 : 0.693146
Execution time for 1000 Iteration 1.5956096649169922
Cost after iteration 1000 : 0.638728
Execution time for 1000 Iteration 876.2019281387329
Cost after iteration 2000 : 0.630216
Execution time for 1000 Iteration 809.2567977905273
Cost after iteration 3000 : 0.613707
Execution time for 1000 Iteration 814.0228354930878
Cost after iteration 4000 : 0.591161
Execution time for 1000 Iteration 807.3985114097595
Cost after iteration 5000 : 0.607249
Execution time for 1000 Iteration 781.5039131641388
Cost after iteration 6000 : 0.577309
Execution time for 1000 Iteration 821.5877752304077
Cost after iteration 7000 : 0.590434
Execution time for 1000 Iteration 769.5549182891846
Cost after iteration 8000 : 0.593967
Execution time for 1000 Iteration 768.3112406730652
Cost after iteration 9000 : 0.549837
Execution time for 1000 Iteration 708.1844613552094


In [17]:
cache=forward_propagation(X_validation,parameters);
A2=cache['A2'];
y_validation_predictions=np.where(A2>=0.5,1,0);
print("test accuracy: {} %".format(100 - np.mean(np.abs(y_validation_predictions - Y_validation)) * 100))



test accuracy: 59.52 %
